In [1]:
import winsound as s
s.MessageBeep()
s.Beep(1300,2000)

In [2]:
import spacy
spacy.load('en')

from spacy.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
#     text.translate(dict.fromkeys(string.punctuation))
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('@'):
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens
s.Beep(1300,2000)

In [3]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Faaiza
[nltk_data]     Iqbal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [5]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [6]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Faaiza
[nltk_data]     Iqbal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [21]:
import random
import re
text_data = []
short_pos = open("dataset/combinedtweet.txt","r",encoding="ANSI").read()
tweet = short_pos.split('\n')
document = []
for t in tweet:
      item = t.split('|')
      if len(item)>=3:
          clean = re.sub(r"http\S+", "", item[2])
          document.append(clean)
print ("Done Cleaning")
for line in document:
    tokens = prepare_text_for_lda(line)
#     if random.random() > .99:
#     print(tokens)
    text_data.append(tokens)
print ("Done...")
s.Beep(1300,2000)

Done Cleaning
Done...


In [22]:
len(text_data)

59817

In [23]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

In [24]:
corpus = [dictionary.doc2bow(text) for text in text_data]

In [25]:
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

# Try 5 topics

In [26]:
import gensim
NUM_TOPICS = 4
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=25)
ldamodel.save('model4.gensim')
s.Beep(1300,2000)

In [27]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.078*"ebola" + 0.025*"patient" + 0.017*"hospital" + 0.013*"doctor" + 0.010*"treatment" + 0.009*"state" + 0.008*"africa" + 0.008*"first" + 0.008*"healthy" + 0.007*"fight"')
(1, '0.039*"study" + 0.016*"heart" + 0.012*"find" + 0.010*"story" + 0.010*"brain" + 0.009*"medicare" + 0.009*"disease" + 0.008*"american" + 0.008*"exercise" + 0.008*"diabetes"')
(2, '0.069*"health" + 0.013*"people" + 0.010*"insurance" + 0.008*"better" + 0.008*"question" + 0.007*"obama" + 0.007*"expert" + 0.007*"recipe" + 0.006*"change" + 0.006*"exchange"')
(3, '0.030*"cancer" + 0.013*"pharma" + 0.011*"report" + 0.011*"drug" + 0.011*"medical" + 0.009*"million" + 0.009*"obamacare" + 0.008*"weight" + 0.007*"death" + 0.007*"breast"')


In [56]:
def predict_class(new_tweet):
    new_tweet = prepare_text_for_lda(new_tweet)
    new_tweet_bow = dictionary.doc2bow(new_tweet)
#     print(new_doc_bow)
    pred =ldamodel.get_document_topics(new_tweet_bow)
#     print (pred)
    max=pred[0][1]
    label=0
    for i in range(1,4):
        if (pred[i][1]>max):
            max= pred[i][1]
            label=i
    return label
#     print (label)
#     print('Label is %d with proabbility of %f' % (label,max))


### pyLDAvis

In [29]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model4.gensim')

In [30]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.show(lda_display)
s.Beep(1300,2000)

C:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8891/    [Ctrl-C to exit]


127.0.0.1 - - [27/May/2018 14:24:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2018 14:24:04] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2018 14:24:04] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2018 14:24:04] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...


In [ ]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

In [32]:
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')
documents_final = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
random.shuffle(documents_final)

[nltk_data] Downloading package movie_reviews to C:\Users\Faaiza
[nltk_data]     Iqbal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


In [139]:
csv = open("dataset/health_tweets_labeled.csv", "a") 

columnTitleRow = "tweet, class\n"
csv.write(columnTitleRow)

for i in range(13618,len(text_data)):
    tweet = text_data[i]
    tweet_str = ' '.join(str(e) for e in tweet)
#     re.sub(r'[^\x00-\x7F]+',' ', tweet_str)
    row = tweet_str.replace(",","") + "," + str(predict_class(tweet_str)) + "\n"
    csv.write(row)
csv.close()

In [138]:
# print(text_data[13617])

['exercise', 'outside', 'simple', 'ã¢â‚¬â€\x9d']


In [140]:
# csv.close()